# Transpilation and Actions


**In this notebook, we are going to go over the instructions on how to use the giza-cli to manage accounts and transpile models. 
Following the transpilation, we are going to use the giza-actions to make the model verifiable!**


*It is assumed that you have succesfully created and trained the liquidation prediction model, and imported it as a .onnx file. If not, go to the aave_liquidations_model.ipynb notebook and run it.*


## 1) Giza Users

Documentation: https://cli.gizatech.xyz/resources/users

Create an account to use the giza cli

```bash
$ giza users create
```

```
Enter your username 😎: ege

Enter your password 🥷 : (this is a secret)

Enter your email 📧: ege@gizatech.xyz

[giza][2024-01-23 12:29:41.417] User created ✅. Check for a verification email 📧
```

Great, after email verification, we can log in!

```bash
$ giza users login
```
```
Enter your username 😎: ege
Enter your password 🥷 :
[giza][2023-07-12 10:52:25.199] Log into Giza
[giza][2023-07-12 10:52:46.998] Credentials written to: /Users/ege/.giza/.credentials.json
[giza][2023-07-12 10:52:47.000] Successfully logged into Giza ✅
```

Lastly, lets create a API-Key

```bash
$ giza users create-api-key
```
```
[giza][2024-01-17 15:27:27.936] Creating API Key ✅ 
[giza][2024-01-17 15:27:53.605] API Key written to: /Users/.....
[giza][2024-01-17 15:27:53.606] Successfully created API Key. It will be used for future requests ✅ 
```

Perfect, we are ready for transpilation!

## 2) Transpilation

Docs: https://cli.gizatech.xyz/resources/models

Now that we are logged in, we can use the cli to transform the .onnx models into verifiable cairo models.

```bash
$ giza transpile model_7day.onnx --output-path liquidation_model_7days
```
```
No model id provided, checking if model exists ✅ 
Model name is: model_7day
Model Created with id -> 294! ✅
Version Created with id -> 1! ✅
Sending model for transpilation ✅ 
Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
Transpilation recieved! ✅
Transpilation saved at: liquidation_model_7days
```

Its that easy! Now, its interesting to take a look at the newly created cairo model folder, where we see the same model we have developed in the previous notebook, but written in Cairo. Its quickly realize however, that there are some differences, such as the model parameters being stored layer-wise in their respective folders. For more information, check out the Orion Docs: https://orion.gizatech.xyz/welcome/readme

![Folder_Structure](images/Screenshot1.png)


Now its time to create a new workspace.

## 3) Workspaces

Docs: https://cli.gizatech.xyz/resources/workspaces

```bash
$ giza workspaces create
```
```
[giza][2024-01-17 14:40:02.046] Creating Workspace ✅ 
[WARNING][2024-01-17 14:40:02.047] This process can take up to 10 minutes ⏳
[giza][2024-01-17 14:41:51.248] Waiting for workspace creation...
[giza][2024-01-17 14:43:12.291] Workspace status is 'PROCESSING'
[giza][2024-01-17 14:45:54.365] Worksace creation is successful ✅
[giza][2024-01-17 14:45:54.366] ✅ Workspace URL: https://actions-server-ege-gageadsga-ew.a.run.app 
```

Great, lets deploy the model!

## 4) Model Deployment

Docs: https://cli.gizatech.xyz/resources/deployments

**The model-id and the version we can easily get from the output of the transpilation**

```bash
$ giza deployments deploy --model-id 294 --version-id 1
```
```
[giza][2024-02-13 16:09:12.428] Deployment is successful ✅
[giza][2024-02-13 16:09:12.429] Deployment created with id -> 21 ✅
[giza][2024-02-13 16:09:12.430] Deployment created with endpoint URL: https://deployment-ege-294-1-2ddbcbbe-7i3yxzspbq-ew.a.run.app 🎉
```

Time to reap the rewards!

## 5) Verifiable Inference

Docs: https://actions.gizatech.xyz/how-to-guides/gizamodel

In [1]:
from giza_actions.action import action
from giza_actions.task import task
from giza_actions.model import GizaModel

In [9]:
import numpy as np

# Dummy Input
file_path = "data_array.npy"

input = np.load(file_path)

print(input.shape)

input = input.reshape(1,7)

print(input.shape)

(7,)
(1, 7)


In [10]:
MODEL_ID = 512  
VERSION_ID = 1  


@task(name=f'wstETH-yield Prediction')
def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, request_id) = model.predict(
        input_feed={"input": input}, verifiable=True)

    return result, request_id


@action(name=f'Execution: wstETH-yield Prediction ', log_prints=True)
def execution():
    (result, request_id) = prediction(input, MODEL_ID, VERSION_ID)
    print("Result: ", result)
    print("Request id: ", request_id)

    return result, request_id


execution()

/Users/egeatesalp/Library/Caches/pypoetry/virtualenvs/yield-predictor-model-AROtystS-py3.11/lib/python3.11/site-packages/prefect/tasks.py:332: UserWarning: A task named 'wstETH-yield Prediction' and defined at '/Users/egeatesalp/Library/Caches/pypoetry/virtualenvs/yield-predictor-model-AROtystS-py3.11/lib/python3.11/site-packages/giza_actions/task.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/egeatesalp/Library/Caches/pypoetry/virtualenvs/yield-predictor-model-AROtystS-py3.11/lib/python3.11/site-packages/prefect/flows.py:336: UserWarning: A flow named 'Execution: wstETH-yield Prediction ' and defined at '/Users/egeatesalp/Library/Caches/pypoetry/virtualenvs/yield-predictor-model-AROtystS-py3.11/lib/python3.11/site-packages/giza_actions/action.py:15' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_un

13:27:57.758 | INFO    | Created flow run 'imported-donkey' for flow 'Execution: wstETH-yield Prediction '

13:27:57.759 | INFO    | Action run 'imported-donkey' - View at https://actions-server-ege-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/d87cf782-73bb-4ea9-910b-1cebb528c949

13:27:58.013 | INFO    | Action run 'imported-donkey' - Created task run 'wstETH-yield Prediction-0' for task 'wstETH-yield Prediction'

13:27:58.014 | INFO    | Action run 'imported-donkey' - Executing 'wstETH-yield Prediction-0' immediately...

...

13:28:02.721 | INFO    | Task run 'wstETH-yield Prediction-0' - Finished in state Completed()

13:28:02.725 | INFO    | Action run 'imported-donkey' - Result:  [[0.05644226]]

13:28:02.726 | INFO    | Action run 'imported-donkey' - Request id:  96e8dbdbe83547a39991217a2ca4398a

13:28:02.824 | INFO    | Action run 'imported-donkey' - Finished in state Completed()

(array([[0.05644226]]), '96e8dbdbe83547a39991217a2ca4398a')